# MiniFlow

In this lesson we will be building a machine learning library called **MiniFlow**. Miniflow will be used as an example to highlight the abstraction principles that **TensorFlow** will providein the next lesson.

## Graphs

A neural net can be thought of a mathematical graph, in which each node is a function, and the edges are the weights.

### Forward Propagation
We have seen plenty of times now what forward propagation is. A set of input values travels forward through then graph and leaves as an output.

A network is created by defining nodes, and propagating values. These are therefore two functions that MiniFlow must posess.

---
##  MiniFlow Architecture

MiniFlow will use Object orientated Python.

Here we will define a Node, and several speclised subclasses.

In [38]:
from functools import reduce

In [59]:
class Node():
    """
    A Generic Base Node.
    """
    
    def __init__(self, inbound_nodes=[]):
        """
        A nodes initalization.
        
        Args:
            inbound_nodes: A list of nodes that input to this node on a forward pass
        """
        self.inbound_nodes  = inbound_nodes        # Where the inputs come from
        self.outbound_nodes = []                   # Where the outputs are going 
        
        for node in self.inbound_nodes:            # Settting the network up
            node.outbound_nodes.append(self)
            
        self.value = None                          # Where the output will be stored
        self.gardient = {}
        

    def forward(self):
        """
        The forward propagation function.
        
        Compute the output value based on `inbound_nodes` and
        store the result in self.value.
        
        Raises:
            NotImplamentedError
        """
        raise NotImplementedError
        
    def backward(self):
        """
        The backward propagation function.
        
        Raises:
            NotImplamentedError
        """
        raise NotImplementedError

In [72]:
class Input(Node):
    """
    An Input Node.
    
    This class of nodes has no inbound nodes.
    """
    
    def __init__(self):
        """
        The Input Nodes initalizaton.
        
        As this class of node has no inbound nodes, then it passes nothing to the superclass,
        as the default there is `[]`
        """
        Node.__init__(self)
    
    def forward(self, value=None):
        """
        The method that initates a forward pass.
        
        This is the only node class that should be passed a value to forward,
        the rest should take it from their inbound_nodes
        """
        if value != None:
            self.value = value
            
    def backward(self):
        self.gardients = {self:0}
        
        for node in self.outbound_nodes:
            other_cost = node.gradients[self]
            self.gradients[self] += grad_cost * 1

In [61]:
class Add(Node):
    """
    An Adding Node.
    """
    
    def __init__(self, *inputs):
        """
        The Add Nodes initalizaton.
        
        As this class of node has no inbound nodes, then it passes nothing to the superclass,
        as the default there is `[]`
        """
        Node.__init__(self, inbound_nodes=inputs)
    
    def forward(self):
        """
        Adds the values of all the input nodes
        
        Sets this nodes value to the sum of that of all input nodes.        
        """
        
        self.value = sum([val.value for val in self.inbound_nodes] )
           

In [62]:
class Mul(Node):
    def __init__(self, *inputs):
        Node.__init__(self, inputs)

    def forward(self):
        """
        For reference, here's the old way from the last
        quiz. You'll want to write code here.
        """
        self.value = reduce(lambda x, y: x*y, [val.value for val in self.inbound_nodes])


Now that we have some of the architecture sorted out, we can begin with think about how they will work

In [71]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.
    
    Args:
        `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns:
        A list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Args:
        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns:
        The output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    for n in graph[::-1]:
        n.backward()

In [64]:
x, y, z = Input(), Input(), Input()

t,u,v = Input(), Input(), Input()

f = Add(x, y, z)
m = Mul(t,u,v)

feed_dict = {x: 4, y: 5, z: 10}

feed_dict_2 = {t: 4, u: 5, v: 10}

graph = topological_sort(feed_dict)
graph2 = topological_sort(feed_dict_2)
output = forward_pass(f, graph)
output_2 = forward_pass(m, graph2)

# should output 19
print("{} + {} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], feed_dict[z], output))
print("{} * {} * {} = {} (according to miniflow)".format(feed_dict_2[t], feed_dict_2[u], feed_dict_2[v], output_2))


4 + 5 + 10 = 19 (according to miniflow)
4 * 5 * 10 = 200 (according to miniflow)


In [74]:
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.

        Your code goes here!
        """
        X = self.inbound_nodes[0].value
        w = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        
        self.value = sum((X[i]*w[i]) for i in range(len(X))) + b
    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            # Set the partial of the loss with respect to this node's inputs.
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # Set the partial of the loss with respect to this node's weights.
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # Set the partial of the loss with respect to this node's bias.
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)


## The Edges Between Layers

The idea of converiting an input to an output in multiple dimensions can be done using Linear Algebra.
A **Transform** does exactly this.

If we go back to the output function of the Linear Node:
$$
y=\sum { w_{ i }x_{ i }\quad  } +\quad b
$$

It is knon that the inuts and the weights are not necisarrily lists, and in fact are most likely to be tensors.
For this reason, we must redefine `Linear` to be able to handel tensors.

In [75]:
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.

        Your code goes here!
        """
        X = self.inbound_nodes[0].value # This is a ndarray
        W = self.inbound_nodes[1].value # This is a ndarray
        b = self.inbound_nodes[2].value # This is a vector
        
        self.value = np.matmul(X,W) + b
    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            # Set the partial of the loss with respect to this node's inputs.
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # Set the partial of the loss with respect to this node's weights.
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # Set the partial of the loss with respect to this node's bias.
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)

## The Sigmoid Function

This Sigmoid Function is amongst the most common activation functions in neural networks for deep learning.

$$
sigmoid(x)\quad =\quad \frac{1}{1+e^{-x}}
$$

Sigmoid mimics the behaviour of the simple step functions that are used in preceptrons, whilst still being (Very easily) differentiable. This trait allows gradient decent to work.

Sigmoid takes a single input, and is an activation function (non-linerarity), meaning that it takes its input, preforms some operation, and produces an output. 



In [79]:
class Sigmoid(Node):
    """
    This sigmoid activation function
    """
    def __init__(self, input_node):
        Node.__init__(self, [input_node])
        
    def _sigmoid(self, X):
        return 1.0/(1.0+np.e**(-X))
    
    def forward(self):
        input_value = self.inbound_nodes[0].value
        self.value  = self._sigmoid(input_value)
    
    def backward(self):
        """
        Calculates the gradient using the derivative of
        the sigmoid function.
        """
        # Initialize the gradients to 0.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}

        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            sigmoid = self.value
            self.gradients[self.inbound_nodes[0]] += sigmoid * (1 - sigmoid) * grad_cost

## Cost, Loss, and Accuracy

These terms are often used somewhat interchangably if refernce to how well a network is doing. We will use **Cost** here. 

In this example we will be using the **Mean Squared Error** as the definition of Cost.

The cost refers to the error that is present as a result of using paticcular weights and biases.

$$
Cost(w,b)\quad = \quad \frac{1}{m}\sum _{ x }^{  }{ { (y(x) - a) }^{ 2 } } 
$$

In [78]:
class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        #m = self.inbound_nodes[0].value.shape[0]       # This is the number of rows of the matrix, its length
                                                        # This is not needed as the np.mean method calculates 
                                                        # it internally  
        
        self.diff = y-a
        
        self.value = np.mean(self.diff**2)
    def backward(self):
        """
        Calculates the gradient of the cost.

        This is the final node of the network so outbound nodes
        are not a concern.
        """
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff
        
        

In [69]:
def gradient_descent_update(x, gradx, learning_rate):
    """
    Performs a gradient descent update.
    """
    return x - learning_rate * gradx

## Backpropagation

We are now ready to implemt back propagation.
To do this, Node has recieved a new method, `backward` as well as another atribute, `self.gradients`, a dictionary that is used to cache gradients.

Another helper function has been added, the `forward_and_backward`. This preforms a forward pass folloed by a backward pass.

Above the backard methods have been implamented.